In [13]:
import openturns as ot
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

random_state = 123
np.random.seed(random_state)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from dependence.tests import func_sum

In [15]:
import skopt

In [16]:
dim = 2
margins = [ot.Normal()]*dim

In [26]:
families = np.zeros((dim, dim), dtype=np.int)
families[1, 0] = 1

In [33]:
from dependence import ConservativeEstimate

quant_estimate = ConservativeEstimate(model_func=func_sum, margins=margins, families=families)

In [38]:
y, x = quant_estimate.stochastic_function(0.1)

In [48]:
f = lambda x: quant_estimate.stochastic_function(x, return_input_sample=False)[0][0]

In [51]:
res = skopt.gp_minimize(f, [(-0.9, 0.9)])

/users/home/benoumechia/anaconda3/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:195: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/users/home/benoumechia/anaconda3/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:195: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


In [53]:
res.x

[-0.03227160834720539]

In [ ]:
res = gp_minimize(f,                  # the function to minimize
                  [(-.99, .99)],      # the bounds on each dimension of x
                  acq_func="EI",      # the acquisition function
                  n_calls=15,         # the number of evaluations of f 
                  n_random_starts=5,  # the number of random initialization points
                  noise=0.1**2,       # the noise level (optional)
                  random_state=123)   # the random seed

In [57]:
from dependence.tests.test_dependence import test_additive_gaussian_gridsearch

In [63]:
QUANTILES_PROB = [0.05, 0.01]
PROB_THRESHOLDS = [1., 2.]
DIMENSIONS = range(2, 4)

In [115]:
test_additive_gaussian_gridsearch()

oui
oui


AssertionError: 
Not equal to tolerance rtol=0.1, atol=0
Failed with alpha = 0.05, dim = 3
(mismatch 40.0%)
 x: array([-1.513435, -4.395331, -2.405256, -3.560215, -1.930969])
 y: array([-1.729682, -4.049006, -2.227923, -3.638882, -1.729682])

In [ ]:
ConservativeEstimate

In [149]:
from dependence.utils import quantile_func
n_params = 5
n_input_sample = 10000
alpha = 0.05
grid = 'lhs'
# Only Gaussian families
families = np.tril(np.ones((dim, dim)), k=1)

impact = ConservativeEstimate(model_func=func_sum,
                          margins=[ot.Normal()]*dim,
                          families=np.ones((dim, dim)))

dep_results = impact.gridsearch_minimize(
n_dep_param=n_params, 
n_input_sample=n_input_sample, 
grid_type=grid, 
random_state=0)

# Quantile results
dep_results.q_func = quantile_func(alpha)
empirical_quantiles = dep_results.quantities


In [150]:
dep_results[0].input_sample.ndim

2

In [151]:
from scipy.stats import gaussian_kde

In [155]:
y = func_sum(dep_results[0].input_sample)

In [161]:
kde = gaussian_kde(y)
q = empirical_quantiles[0]

-2.3406132712052004

In [165]:
dep_results[0]

In [175]:
def q_var(n):
    tmp = alpha * (1. - alpha) / (n * kde.pdf(q))
    return np.sqrt(tmp)

q_var(100000)

array([ 0.0025586])